In [2]:
import numpy as np
import time

In [91]:
class TM():
    def __init__(self):
        self.tape = np.zeros(35, dtype=np.int8)
        self.add_table = np.array([[[0, 0], [1, 0], [2, 0]], 
                                   [[1, 0], [2, 0], [0, 1]], 
                                   [[2, 0], [0, 1], [1, 1]]])
        
        self.sub_table = np.array([[[0, 0], [1, 0], [2, 0]], 
                                   [[2, 1], [0, 0], [1, 0]], 
                                   [[1, 1], [2, 1], [0, 0]]])
        
        self.mul_table = np.array([[[0, 0], [0, 0], [0, 0]], 
                                   [[0, 0], [1, 0], [2, 0]], 
                                   [[0, 0], [2, 0], [1, 1]]])
        self.q = 0
        self.movements = 0
        self.total_movements = 0 

    def __digit_add(self, digit, adress):
        self.q = digit
        a = self.tape[adress]
        if a == 0 and self.q == 0:
            pass
        else:
            a, self.q = self.add_table[self.q, a]
            self.tape[adress] = a
            self.movements += 1
            self.__digit_add(self.q, adress-1)
            
    def __digit_subtract(self, digit, adress):
        self.q = digit
        a = self.tape[adress]
        if a == 0 and self.q == 0:
            pass
        else:
            a, self.q = self.sub_table[self.q, a]
            
            self.tape[adress] = a
            self.total_movements += 1
            self.__digit_subtract(self.q, adress-1)
            
    def __digit_multiply(self, digit, adress):
        if adress >= len(self.tape):
            pass
        else:
            self.q = digit
            a = self.tape[adress]
            a, self.q = self.mul_table[self.q, a]
            self.tape[adress] = a
            self.__digit_add(self.q, adress-1)
            self.movements += len(self.tape) - adress
            self.__digit_multiply(digit, adress+1)
            
    def left_mov(self):
        self.tape[0:len(self.tape)-1] = self.tape[1:len(self.tape)]
        self.tape[len(self.tape)-1] = 0
        self.total_movements += 2*(len(self.tape)-1)*len(self.tape)
            
    def multiply(self, number):
        r = 3
        l = len(self.tape)
        digits = []
        length_number = 0
        feed = 0
        inner_tm = TM()
        while number != 0:
            feed = number%r
            number = (number - feed)/r
            digits.append(int(feed))
            length_number += 1
            
        mem_storage = np.copy(self.tape)
        self.__digit_multiply(0, 0)
        
        i = 0
        for d in digits:
            inner_tm.tape = np.copy(mem_storage)
            self.total_movements += len(self.tape)**2
            inner_tm.__digit_multiply(d, 0)
            for j in range(i):
                inner_tm.left_mov()
            result = inner_tm.get_number()
            self.add(result)
            i += 1
            
        self.total_movements += inner_tm.get_mov()
            
            
    def add(self, number):
        r = 3
        i = 0
        l = len(self.tape)
        length_number = 0
        feed = 0
        if number == 0:
            length_number = 1
        while number != 0:
            feed = number%r
            number = (number - feed)/r
            self.movements += 1
            self.__digit_add(int(feed), l - 1 - i)
            i += 1
            length_number += 1
        self.movements += np.math.factorial(length_number-1)
        self.total_movements += self.movements
        self.movements = 0
        
    def subtract(self, number):
        r = 3
        i = 0
        l = len(self.tape)
        length_number = 0
        feed = 0
        while number != 0:
            feed = number%r
            number = (number - feed)/r
            self.movements += 1
            self.__digit_subtract(int(feed), l - 1 - i)
            i += 1
            length_number += 1
        #self.movements += np.math.factorial(length_number-1)
        self.total_movements += self.movements
        self.movements = 0
        
            
    def mem_write(self, number):
        r = 3
        i = 0
        l = len(self.tape)
        while number != 0:
            self.tape[len(self.tape) - i - 1] = number%r
            number = (number - self.tape[l - i - 1])/r
            i+=1
        for j in range(i, len(self.tape)):
            self.tape[len(self.tape) - i - 1 - j] = 0
            
    def get_number(self):
        term = 0
        for i in range(len(self.tape)):
            if self.tape[len(self.tape) - 1 - i] != 0:
                term += (3**i)*self.tape[len(self.tape) - 1 - i]
        return term
    
    def display(self):
        print(self.tape)
        
    def mov_count(self):
        print('movs >= ', self.total_movements)
        
    def get_mov(self):
        return self.total_movements
            



In [100]:
m1 = TM()

#write - записать число на ленту, display - отобразить ленту, getnumber - перевести число из троичной СС в десятичную
#add, subtract, multiply - операции сложения, вычитания и умножения соотвесттвенно, mov_count колчиетсво переходов, 
#выполнененных во время выполнения запроса
m1.mem_write(30)  
m1.display()
m1.add(48)
m1.display()
m1.subtract(10)
m1.mov_count()
m1.display()
m1.multiply(68)
m1.display()
print(m1.get_number())
m1.mov_count()

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 0]
movs >=  26
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 1 1 2]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 1 0 0 0 2 1]
4624
movs >=  25112
